In [6]:
from mp_api.client import MPRester
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

API_KEY = "4IeHY5jVcrgiKXNuAo6Jgs7yC0Z3hsli"

with MPRester(API_KEY) as mpr:
    # Pull summary data
    materials = mpr.materials.insertion_electrodes.search(
            working_ion="Li",
            fields=[
                "battery_id", "battery_formula", "formula_discharge", "average_voltage"
                , "energy_grav", "capacity_grav", "stability_charge", "fracA_charge", "max_delta_volume"
            ]
        )

# Convert to DataFrame
mp_df = pd.DataFrame([{
    "battery_id": m.battery_id,
    "battery_formula": m.battery_formula,
    "average_voltage": m.average_voltage,
    "capacity_grav": m.capacity_grav,
    "energy_grav": m.energy_grav,
    "max_delta_volume": m.max_delta_volume,
    "fracA_charge": m.fracA_charge,
    "stability_charge": m.stability_charge,
} for m in materials])

# Drop any rows with missing values (critical)
mp_df = mp_df.dropna().reset_index(drop=True)

# Verify data exists
print("Number of materials pulled:", len(mp_df))
if len(mp_df) == 0:
    raise ValueError("No materials data available after dropping missing values!")

# -------------------------------
# Normalize and prepare features
# -------------------------------
features = ["average_voltage",
    "capacity_grav",
    "energy_grav",
    "max_delta_volume",
    "fracA_charge",
    "stability_charge"]
scaler = MinMaxScaler()
mp_df[features] = scaler.fit_transform(mp_df[features])

# Invert features where lower is better
for col in ["average_voltage",
    "capacity_grav",
    "energy_grav",
    "max_delta_volume",
    "fracA_charge",
    "stability_charge"]:
    mp_df[col] = 1 - mp_df[col]

# Compute composite intrinsic score
mp_df["intrinsic_score"] = mp_df[features].mean(axis=1)

# -------------------------------
# Rank and select top candidates
# -------------------------------
top_candidates = mp_df.sort_values("intrinsic_score", ascending=False).head(20)

# Print to check
print("Top candidates based on intrinsic score:")
print(top_candidates[["battery_id", "battery_formula", "intrinsic_score"]])


Retrieving InsertionElectrodeDoc documents: 100%|██████████| 2774/2774 [00:00<00:00, 7514.48it/s]

Number of materials pulled: 2774
Top candidates based on intrinsic score:
         battery_id   battery_formula  intrinsic_score
1062   mp-557663_Li     Li0-0.5MnB4O7         0.956446
1028   mp-510369_Li    Li0-0.5Ce5Pb3O         0.955263
2245    mp-23637_Li    Li0-0.5VBiPbO5         0.954223
1555   mp-756679_Li     Li0-1Mn9CdO10         0.952739
2289   mp-559270_Li  Li0-0.5RbBa4Sb3O         0.952067
488     mp-13093_Li    Li0-2La3Ni2Sn7         0.951833
2239    mp-18794_Li  Li0-0.5Sm2TiMnO6         0.951515
996     mp-36866_Li    Li0-0.5Al2CdO4         0.951436
2240    mp-19194_Li   Li0-0.5BaMnP2O7         0.951306
2223  mp-1218828_Li     Li0-1Sr2FeO3F         0.950831
467   mp-1188225_Li     Li0-0.25SmVO3         0.950741
1873   mp-762225_Li      Li0-0.5BaCO3         0.950467
2023   mp-772222_Li      Li0-0.5TmClO         0.950296
39    mp-1106357_Li       Li0-1Tb5Sn3         0.950097
2284   mp-558128_Li   Li0-0.5KNd2NbO6         0.949669
1507   mp-756147_Li    Li0-0.25CrInO3         

In [2]:
print(mpr.materials.insertion_electrodes.available_fields)

['builder_meta', 'battery_type', 'battery_id', 'thermo_type', 'battery_formula', 'working_ion', 'num_steps', 'max_voltage_step', 'last_updated', 'framework', 'framework_formula', 'elements', 'nelements', 'chemsys', 'formula_anonymous', 'warnings', 'formula_charge', 'formula_discharge', 'max_delta_volume', 'average_voltage', 'capacity_grav', 'capacity_vol', 'energy_grav', 'energy_vol', 'fracA_charge', 'fracA_discharge', 'stability_charge', 'stability_discharge', 'id_charge', 'id_discharge', 'host_structure', 'adj_pairs', 'material_ids', 'entries_composition_summary', 'electrode_object']
